Convnet stuff
--
I am planning t us tflearn which is a wrapper around tensorflow

In [1]:
import tflearn

In [2]:
# Load HDF5 dataset
import h5py
h5f = h5py.File('traindataset.h5', 'r')
X_train_images = h5f['X']
Y_train_labels = h5f['Y']

h5f2 = h5py.File('valdataset.h5', 'r')
X_val_images = h5f2['X']
Y_val_labels = h5f2['Y']

In [3]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [4]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [5]:
# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

In [6]:
# Input is a 50x50 image with 1 color channels (grayscale)
network = input_data(shape=[None, 50, 50, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

In [7]:
# Step 1: Convolution
network = conv_2d(network, 50, 3, activation='relu')

In [8]:
# Step 2: Max pooling
network = max_pool_2d(network, 2)

In [9]:
# Step 3: Convolution again
network = conv_2d(network, 64, 3, activation='relu')

In [10]:
# Step 4: Convolution yet again
network = conv_2d(network, 64, 3, activation='relu')

In [11]:
# Step 5: Max pooling again
network = max_pool_2d(network, 2)

In [12]:
# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

In [13]:
# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

In [14]:
# Step 8: Fully-connected neural network with two outputs (0=isn't a nodule, 1=is a nodule) to make the final prediction
network = fully_connected(network, 2, activation='softmax')

In [15]:
# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

In [16]:
# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='nodule-classifier.tfl.ckpt')

In [17]:
# Train it! We'll do 100 training passes and monitor it as it goes.
model.fit(X_train_images, Y_train_labels, n_epoch=100, shuffle=True, validation_set=(X_val_images, Y_val_labels),
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
          run_id='nodule-classifier')

Training Step: 7199  | time: 6.320s
| Adam | epoch: 100 | loss: 0.00000 - acc: 1.0000 -- iter: 6816/6877
Training Step: 7200  | time: 7.424s
| Adam | epoch: 100 | loss: 0.00000 - acc: 1.0000 | val_loss: 18.32126 - val_acc: 0.2043 -- iter: 6877/6877
--
INFO:tensorflow:/home/a_khanss/dl/pattern-recognition/nodule-classifier.tfl.ckpt-7200 is not in all_model_checkpoint_paths. Manually adding it.


In [18]:
# Save model when training is complete to a file
model.save("nodule-classifier.tfl")
print("Network trained and saved as nodule-classifier.tfl!")

INFO:tensorflow:/home/a_khanss/dl/pattern-recognition/nodule-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
Network trained and saved as nodule-classifier.tfl!


In [20]:
h5f2 = h5py.File('testdataset.h5', 'r')
X_test_images = h5f2['X']
Y_test_labels = h5f2['Y']

In [21]:
model.predict(X_test_images)

OSError: Can't prepare for writing data (No write intent on file)